In [11]:
from langchain.agents import AgentExecutor,  tool, create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

# 1. Create the model and memory:
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
memory = ConversationBufferMemory(return_messages=True)


@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


# 2. Create the tools:
tools = [get_word_length]

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 3. Create the Prompt:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events and isn't good at calculating word length.",
        ),
        # This is where the agent will write/read its messages from
        MessagesPlaceholder(variable_name="agent_scratchpad"),
        MessagesPlaceholder(variable_name="history"),
        ("user", "{input}"),
    ]
)

In [13]:
from langchain_community.tools.convert_to_openai import format_tool_to_openai_function
from langchain_core.runnables import RunnableLambda
from operator import itemgetter

# 4. Formats the python function tools into JSON schema and binds them to the model:
llm_with_tools = model.bind(functions=[format_tool_to_openai_function(t) for t in tools])

from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

# 5. Setting up the agent chain:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
        "history": RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

In [14]:
# Construct the OpenAI functions agent:
agent = create_openai_functions_agent(llm_with_tools, tools, prompt)

In [16]:
# Create an agent executor by passing in the agent, tools and memory:
memory = ConversationBufferMemory(return_messages=True)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

In [18]:
agent_executor.invoke({"input": "My name is James"})



> Entering new AgentExecutor chain...
Hello James! How can I assist you today?

> Finished chain.


{'input': 'My name is James',
 'history': [HumanMessage(content='My name is James'),
  AIMessage(content='Hello James! How can I assist you today?')],
 'output': 'Hello James! How can I assist you today?'}

In [19]:
agent_executor.invoke({"input": "What are the first three characters of my name?"})



> Entering new AgentExecutor chain...
The first three characters of your name, James, are "Jam".

> Finished chain.


{'input': 'What are the first three characters of my name?',
 'history': [HumanMessage(content='My name is James'),
  AIMessage(content='Hello James! How can I assist you today?'),
  HumanMessage(content='What are the first three characters of my name?'),
  AIMessage(content='The first three characters of your name, James, are "Jam".')],
 'output': 'The first three characters of your name, James, are "Jam".'}